# Identication of putative contaminant taxa

In [1]:
setwd("/mnt/c/Users/Cedric/Desktop/git_repos/blood_microbiome")
require(foreach)
require(tidyverse)
require(ggplot2)
require(data.table)
require(doParallel)
require(compositions)
require(VennDiagram)
registerDoParallel(cores=6)

Loading required package: foreach

Loading required package: tidyverse

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ purrr::accumulate() masks foreach::accumulate()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::lag()        masks stats::lag()
✖ purrr::when()       masks foreach::when()

Loading required package: data.table


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose


Loading required package: doPara

### Load and preprocess data

In [2]:
load_metadata <- function(file_path, df) {
    meta <- fread(file_path, na.strings=c("", NA))
    meta <- meta %>% 
        filter(npm_research_id %in% df$npm_research_id) %>%
        select(-removal_requested_by_supplier) %>%
        replace(is.na(.), "unknown")
    return(meta)
}


get_meta_cols <- function(meta, meta_regex, to_exclude) {
    meta_cols <- colnames(meta)[grep(meta_regex, colnames(meta))]
    meta_cols <- meta_cols[!(meta_cols %in% to_exclude)]
    return(meta_cols)
}

In [3]:
corr_t <- 0.7

# Load data and remove low microbial read samples
to_retain <- fread("data/samples_above_100_microbial_reads.txt")$npm_research_id

prev_df <- fread("results/decontamination/prevalence_RA0.005_read10.csv")
corr_nc <- read.csv(str_glue("results/decontamination/correlation_decontamination/nc.diff_prev_V3.RA0.005.read_threshold10.max_prev0.25.fold_diff2.corr_t{corr_t}.S.n9999.txt"))$non_contaminant_taxon
prev_df <- prev_df %>% select(all_of(c("npm_research_id", corr_nc)))
prev_df

meta <- load_metadata("data/SG10K_Health_metadata.n10714.16March2021.parsed.csv", prev_df) %>% 
    filter(npm_research_id %in% to_retain)

# Get metadata columns of interest
meta_cols <- get_meta_cols(meta, 
                           meta_regex = "kit|flow_cell|site_supplying", 
                           to_exclude = c("library_prep_kit", "hiseq_xtm_flow_cell_v2_5_id"))


npm_research_id,Abiotrophia defectiva,Acidaminococcus intestini,Acidovorax sp. 1608163,Acidovorax sp. JMULE5,Acidovorax sp. KKS102,Acinetobacter baumannii,Acinetobacter bereziniae,Acinetobacter dispersus,Acinetobacter guillouiae,⋯,Veillonella nakazawae,Veillonella parvula,Vibrio anguillarum,Woolly monkey hepatitis B virus,Xanthomonas campestris,Xanthomonas euvesicatoria,Xylanimicrobium sp. FW10M-9,Xylanimonas allomyrinae,Xylanimonas cellulosilytica,Zymoseptoria tritici
<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
WHB8690,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
WHB10005,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
WHB4505,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
WHB7030,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
WHB8628,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
WHB4900,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
WHH1727,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
WHB3762,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
0616-0028,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE


In [4]:
# column <- "extraction_kit"
result_df <- tibble(taxa = corr_nc)

for(column in meta_cols) {
    res <- prev_df %>% 
        left_join(meta %>% select(all_of(c("npm_research_id", column)))) %>%
        select(-npm_research_id) %>%
        pivot_longer(!column, names_to = "taxa", values_to = "presence") %>%
        filter(presence) %>%
        group_by(taxa) %>%
        summarise(n_batches = n_distinct(get(column)))
    colnames(res)[2] <- column

    result_df <- result_df %>%
        left_join(res, "taxa")
}

result_df <- result_df %>% column_to_rownames("taxa")
result_df[result_df <= 1] <- 0
result_df[result_df > 1] <- 1
result_df

Joining, by = "npm_research_id"

Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(column)` instead of `column` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.

Joining, by = "npm_research_id"

Joining, by = "npm_research_id"

Joining, by = "npm_research_id"

Joining, by = "npm_research_id"

Joining, by = "npm_research_id"

Joining, by = "npm_research_id"



,site_supplying_sample,extraction_kit,hiseq_xtm_sbs_kit_300_cycles_v2__box_1of_2__lot,hiseq_xtm_sbs_kit_300_cycles_v2__box_2_of_2__lot,hiseq_xtm_pe_cluster_kit_cbottm_v2__box_1_of_2__lot,hiseq_xtm_pe_cluster_kit_cbottm_v2__box_2_of_2__lot,hiseq_xtm_flow_cell_v2_5_lot
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Abiotrophia defectiva,0,0,0,0,0,0,0
Acidaminococcus intestini,0,0,1,1,1,1,1
Acidovorax sp. 1608163,1,1,1,1,1,1,1
Acidovorax sp. JMULE5,1,1,1,1,1,1,1
Acidovorax sp. KKS102,1,1,1,1,1,1,1
Acinetobacter baumannii,1,1,1,1,1,1,1
Acinetobacter bereziniae,0,0,1,1,0,1,0
Acinetobacter dispersus,1,1,1,1,1,1,1
Acinetobacter guillouiae,1,1,1,1,1,1,1


#### Parse results

In [5]:
row_sums <- rowSums(result_df)
parsed_df <- tibble(taxa = names(row_sums), n_cols = row_sums)

In [6]:
prev_stats <- apply(prev_df[, 2:ncol(prev_df)], 2, sum) / nrow(prev_df)
overall_prev <- data.frame(taxa = names(prev_stats), overall_prevalence = as.vector(prev_stats)) %>%
    mutate(n_samples = overall_prevalence * nrow(prev_df))

parsed_df %>% 
    left_join(overall_prev) %>%
    filter(n_cols == 7) %>%
    arrange(desc(overall_prevalence))

nc <- parsed_df %>%
    filter(n_cols == 7) %>%
    select(taxa)

fwrite(nc, str_glue("results/decontamination/simple_batch_decontam/nc.corr_t{corr_t}.txt"))

Joining, by = "taxa"



taxa,n_cols,overall_prevalence,n_samples
<chr>,<dbl>,<dbl>,<dbl>
Cutibacterium acnes,7,0.047462619,419
Microbacterium sp. PM5,7,0.035681921,315
Pseudomonas tolaasii,7,0.024014499,212
Mesorhizobium sp. Pch-S,7,0.019143634,169
Moraxella osloensis,7,0.019143634,169
Bradyrhizobium arachidis,7,0.018690530,165
Human mastadenovirus C,7,0.018577254,164
Paraburkholderia sp. 7MH5,7,0.015745356,139
Paraburkholderia tropica,7,0.014839148,131
